<a href="https://colab.research.google.com/github/Matveycher/3T-2024/blob/master/1colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from colabcode import ColabCode
ColabCode(port=10000)

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, MaxAbsScaler
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model

# Load the data
df = pd.read_csv("train.csv")

# Extract features and target variable
X = df.drop("time_remaining", axis=1)
y = df["time_remaining"]

# Define categorical and numerical columns
categorical_cols = ["firmware_version", "drone_model_number", "payload_type", "country", "drone_color", "flight_mode", "manufacturer_name"]
numerical_cols = [col for col in X.columns if col not in categorical_cols]

# Preprocess date column
X['manufacturing_date'] = pd.to_datetime(X['manufacturing_date'])
reference_date = X['manufacturing_date'].min()
X['manufacturing_date'] = (X['manufacturing_date'] - reference_date).dt.days

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(X[categorical_cols]).toarray()
X_encoded = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical_cols))

# Normalize numerical features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X[numerical_cols])
X_scaled = pd.DataFrame(X_scaled, columns=numerical_cols)

# Concatenate encoded categorical features and normalized numerical features
X_processed = pd.concat([X_encoded, X_scaled], axis=1)

# Define input layers for each feature
inputs = []
for i in range(X_processed.shape[1]):
    input_layer = Input(shape=(1,), name=f"input_{i}")
    inputs.append(input_layer)

# Concatenate all input layers
concatenated = concatenate(inputs)

# Dense layers for processing concatenated inputs
x = Dense(64, activation='relu')(concatenated)
x = Dense(32, activation='relu')(concatenated)

# Output layer
output = Dense(1)(x)

# Create the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Compile the model
# model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
# model.fit([X_processed.iloc[:, i] for i in range(X_processed.shape[1])], y, epochs=500, batch_size=10, validation_split=0.2)
# model.fit(X_processed, y, epochs=500, batch_size=10, validation_split=0.2)

model.fit(X_train, y_train)


# Make predictions
predictions = model.predict([X_processed.iloc[:, i] for i in range(X_processed.shape[1])])
